**AI Agent using CrewAI and Amazon Nova Premier**

Use case : AI Candidate Screening.

This agent help you analyze CV against job requirements, give result for the next step of the recruitment process and create interview question.

In [ ]:
# Install CrewAI and AWS
!pip install python-dotenv packaging crewai pypdf boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 18.8 MB/s e

In [ ]:
import os
# Load environment and save AWS token
from dotenv import load_dotenv
load_dotenv("awskey.txt")
# Get AWS access key, AWS secret key and AWS region
accesskey = os.getenv("AWS_ACCESS_KEY")
secretkey = os.getenv("AWS_SECRET_KEY")
region = os.getenv("AWS_REGION")

In [ ]:
# Import CrewAI
from crewai import LLM, Agent, Task, Crew, Process

llm = LLM(
    model="bedrock/us.amazon.nova-premier-v1:0",
    temperature=0.0,
    aws_access_key_id=accesskey,
    aws_secret_access_key=secretkey,
    aws_region_name=region
)

**Structure of this AI Agent**

**User --> Extract CV --> Compare and Match between Requirements and CV --> Result to the next step of the recruitment process --> Create Interview Question --> End**

**Create CrewAI Agent**

In [ ]:
candidateScreeningAgent = Agent(
    role='Candidate Screening Agent',
    goal='Compare and match between job requirements and extracted of curriculum vitae then give result for next step of recruitment process.',
    backstory="""
    Compare match agent that make sure can compare and match between job requirements and extracted of curriculum vitae and give value from 0 until 10 to the next step of recruitment process.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm
)

**Create CrewAI Task**

In [ ]:
candidateScreeningTask = Task(
    description="""
    You are a virtual human resources expert. You help me COMPARE and MATCH between {cv_data} and requirements like this :
    1. Work across the AI lifecycle: from data preparation and model development to evaluation and deployment.
    2. Fine-tune and integrate LLMs (like OpenAI and Claude) into ERP workflows.
    3. Build smart features such as recommendation engines, forecasting modules, NLP tools, and more.
    Preferred requirements :
    Develop and maintain scalable cloud-based AI solutions across multi-cloud platforms (AWS, GCP, Azure).
    After you compare and match, give value from 0 until 10 to the next step of recruitment process.
    After give value, create 3 interview questions about CV PDF file that already extracted.
    BUT if result are is LESS THAN 7 of 10, don't create interview questions.
    Otherwise, create 3 interview questions about CV PDF file that already extracted.
    Write output structure MUST like example this :
    """,
    expected_output="""
    MINIMAL REQUIREMENTS ANALYSIS :

    Requirement 1:

    Requirement n (n is the number of requirement):

    PREFERRED REQUIREMENTS ANALYSIS :

    OVERALL MATCH PERCENTAGE :

    Strengths:

    Potential Gaps:

    RESULT : ..... of 10

    QUESTION 1 : .....

    QUESTION 2 : .....

    QUESTION 3 : .....
    """,
    agent=candidateScreeningAgent
)

**Create CrewAI Crew**

In [ ]:
candidate_screening_crew = Crew(
    agents=[candidateScreeningAgent],
    tasks=[candidateScreeningTask],
    verbose=True,
)

**Try upload first CV PDF file**

In [ ]:
# Extract CV PDF file
from pypdf import PdfReader
reader = PdfReader("Always Winner CV.pdf")
page = reader.pages[0]
text = page.extract_text()
print(text)

Name  :  Always  Winner  
Phone  Number  :  000888999  
Email  Address  :  alwayswinner@gmail.com 
Work  Experience  :   
AI  Engineer,  Winner,  LLC  |  New  York  City,  NY  |  January  1,  2025  –  Present  
Junior  AI  Engineer,  Nerwin,  LLC  |  New  York  City,  NY  |  January  1,  2024  –  December  31,  2024  
Education  :   
Bachelor  of  Artificial  Intelligence  |  University  of  AI  Everywhere  |  New  York  City,  NY  |  2023  
Technical  Skills  :   
●  Programming  Languages:  Python  (TensorFlow,  PyTorch,  Keras,  scikit-learn,  Hugging  
Face
 
Transformers),
 
Java,
 
C++
 ●  Generative  AI:  GANs  (StyleGAN,  BigGAN),  VAEs,  Diffusion  Models,  Autoregressive  
Models,
 
LLMs
 
(GPT,
 
BERT,
 
LLaMA),
 
Stable
 
Diffusion
 ●  Agentic  AI  ●  Machine  Learning:  Supervised  Learning,  Unsupervised  Learning,  Transfer  Learning,  
Model
 
Optimization,
 
Hyperparameter
 
Tuning
 ●  Cloud  Platforms:  AWS  (Sagemaker,  EC2,  S3),  Azure  (Azure  ML),  Google  Cloud 

In [ ]:
result = candidate_screening_crew.kickoff(inputs={'cv_data': text})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a75f7074-9bcb-4585-8710-fd3d6558ea07                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Candidate Screening Agent                                                                               │
│                                                                                                                 │
│  Task:                                                                                                          │
│      You are a virtual human resources expert. You help me COMPARE and MATCH between Name  :  Always  Winner    │
│  Phone  Number  :  000888999                                                                                    │
│  Email  Address  :  alwayswinner@gmail.com                                                                      │
│  Work  Experience  :                                                                                            │
│  AI  Engineer,  Winner,  LLC  |  New  York  City,  NY  |  January  1,  2025  –  Present                         │
│  Junior  AI  Engineer,  Nerwin,  LLC  |  New  York  City,  NY  |  January  1,  2024  –  December  31,  2024     │
│  Education  :                                                                                                   │
│  Bachelor  of  Artificial  Intelligence  |  University  of  AI  Everywhere  |  New  York  City,  NY  |  2023    │
│  Technical  Skills  :                                                                                           │
│  ●  Programming  Languages:  Python  (TensorFlow,  PyTorch,  Keras,  scikit-learn,  Hugging                     │
│  Face                                                                                                           │
│                                                                                                                 │
│  Transformers),                                                                                                 │
│                                                                                                                 │
│  Java,                                                                                                          │
│                                                                                                                 │
│  C++                                                                                                            │
│   ●  Generative  AI:  GANs  (StyleGAN,  BigGAN),  VAEs,  Diffusion  Models,  Autoregressive                     │
│  Models,                                                                                                        │
│                                                                                                                 │
│  LLMs                                                                                                           │
│                                                                                                                 │
│  (GPT,                                                                                                          │
│                                                                                                                 │
│  BERT,                                                                                                          │
│                                                                                                                 │
│  LLaMA),                                                                                                        │
│                                                                                                                 │
│  Stable                                                                                                         │
│                                                        

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Candidate Screening Agent                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  MINIMAL REQUIREMENTS ANALYSIS :                                                                                │
│                                                                                                                 │
│  Requirement 1: The candidate has experience as an AI Engineer and Junior AI Engineer, demonstrating            │
│  familiarity with the AI lifecycle.                                                                             │
│                                                                                                                 │
│  Requirement 2: The candidate lists LLMs (e.g., GPT, BERT) in their technical skills, showing capability in     │
│  fine-tuning and integrating these models.                                                                      │
│                                                                                                                 │
│  Requirement 3: Skills in machine learning and NLP tools align with building recommendation engines and         │
│  forecasting modules.                                                                                           │
│                                                                                                                 │
│  PREFERRED REQUIREMENTS ANALYSIS :                                                                              │
│                                                                                                                 │
│  The candidate has experience with AWS, Azure, and Google Cloud Platform, indicating they can develop scalable  │
│  cloud-based AI solutions.                                                                                      │
│                                                                                                                 │
│  OVERALL MATCH PERCENTAGE :                                                                                     │
│                                                                                                                 │
│  Strengths: Extensive experience in AI roles, strong technical skills in required areas, and familiarity with   │
│  multi-cloud platforms.                                                                                         │
│                                                                                                                 │
│  Potential Gaps: No explicit mention of ERP workflows, but technical foundation suggests adaptability.          │
│                                                                                                                 │
│  RESULT : 9 of 10                                                                                               │
│                                                                                                                 │
│  QUESTION 1 : Can you describe a project where you integrated LLMs into an existing system or workflow?         │
│                                                                                                                 │
│  QUESTION 2 : How have you approached building and deploying recommendation engines in your previous roles?     │
│                                                                                                                 │
│  QUESTION 3 : What strategies do you use for optimizing

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ea3357ab-c8fb-41b4-97f7-3fbfe5101034                                                                     │
│  Agent: Candidate Screening Agent                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a75f7074-9bcb-4585-8710-fd3d6558ea07                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: MINIMAL REQUIREMENTS ANALYSIS :                                                                  │
│                                                                                                                 │
│  Requirement 1: The candidate has experience as an AI Engineer and Junior AI Engineer, demonstrating            │
│  familiarity with the AI lifecycle.                                                                             │
│                                                                                                                 │
│  Requirement 2: The candidate lists LLMs (e.g., GPT, BERT) in their technical skills, showing capability in     │
│  fine-tuning and integrating these models.                                                                      │
│                                                                                                                 │
│  Requirement 3: Skills in machine learning and NLP tools align with building recommendation engines and         │
│  forecasting modules.                                                                                           │
│                                                                                                                 │
│  PREFERRED REQUIREMENTS ANALYSIS :                                                                              │
│                                                                                                                 │
│  The candidate has experience with AWS, Azure, and Google Cloud Platform, indicating they can develop scalable  │
│  cloud-based AI solutions.                                                                                      │
│                                                                                                                 │
│  OVERALL MATCH PERCENTAGE :                                                                                     │
│                                                                                                                 │
│  Strengths: Extensive experience in AI roles, strong technical skills in required areas, and familiarity with   │
│  multi-cloud platforms.                                                                                         │
│                                                                                                                 │
│  Potential Gaps: No explicit mention of ERP workflows, but technical foundation suggests adaptability.          │
│                                                                                                                 │
│  RESULT : 9 of 10                                                                                               │
│                                                                                                                 │
│  QUESTION 1 : Can you describe a project where you integrated LLMs into an existing system or workflow?         │
│                                                                                                                 │
│  QUESTION 2 : How have you approached building and deploying recommendation engines in your previous roles?     │
│                                                       

In [ ]:
print(result)

MINIMAL REQUIREMENTS ANALYSIS :

Requirement 1: The candidate has experience as an AI Engineer and Junior AI Engineer, demonstrating familiarity with the AI lifecycle.

Requirement 2: The candidate lists LLMs (e.g., GPT, BERT) in their technical skills, showing capability in fine-tuning and integrating these models.

Requirement 3: Skills in machine learning and NLP tools align with building recommendation engines and forecasting modules.

PREFERRED REQUIREMENTS ANALYSIS :

The candidate has experience with AWS, Azure, and Google Cloud Platform, indicating they can develop scalable cloud-based AI solutions.

OVERALL MATCH PERCENTAGE :

Strengths: Extensive experience in AI roles, strong technical skills in required areas, and familiarity with multi-cloud platforms.

Potential Gaps: No explicit mention of ERP workflows, but technical foundation suggests adaptability.

RESULT : 9 of 10

QUESTION 1 : Can you describe a project where you integrated LLMs into an existing system or workflow?

**Try upload second CV PDF file**

In [ ]:
# Extract CV PDF file
from pypdf import PdfReader
reader = PdfReader("Sonny Wawwak CV.pdf")
page = reader.pages[0]
texted = page.extract_text()
print(texted)

Sonny  Wawwak  
UI/UX
 
Designer
 
Address
 
:
 
Konoha
 
Phone
 
:
 
+62
 
812-3456-7890
 
Email
 
Address
 
:
 
sonny.wawwak@email.com
 
Skills  ●  UI  Design  (Web  &  Mobile)  ●  UX  Research  &  Analysis  ●  Wireframing  &  Prototyping  ●  Design  Systems  &  Style  Guides  ●  User  Flows  &  Journey  Mapping  ●  Usability  Testing  ●  Tools:  Figma,  Adobe  XD,  Sketch,  InVision,  Photoshop,  Illustrator  ●  HTML,  CSS  (basic  understanding)  ●  Agile  &  Scrum  Workflow  
Professional  Experience  UI/UX  Designer  
PT
 
I
 
Don’t
 
Know
 
–
 
Konoha
 
Jan
 
2022
 
–
 
Present
  Junior  UI/UX  Designer  
Know
 
Money
 
Only
 
Studio
 
–
 
Konoha
 
Aug
 
2020
 
–
 
Dec
 
2021
 
Education  
Bachelor’s  Degree  in  Visual  Communication  Design  
Design
 
Institute
 
-
 
Konoha
 
2016
 
–
 
2020
 


In [ ]:
result_2 = candidate_screening_crew.kickoff(inputs={'cv_data': texted})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a75f7074-9bcb-4585-8710-fd3d6558ea07                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Candidate Screening Agent                                                                               │
│                                                                                                                 │
│  Task:                                                                                                          │
│      You are a virtual human resources expert. You help me COMPARE and MATCH between Sonny  Wawwak              │
│  UI/UX                                                                                                          │
│                                                                                                                 │
│  Designer                                                                                                       │
│                                                                                                                 │
│  Address                                                                                                        │
│                                                                                                                 │
│  :                                                                                                              │
│                                                                                                                 │
│  Konoha                                                                                                         │
│                                                                                                                 │
│  Phone                                                                                                          │
│                                                                                                                 │
│  :                                                                                                              │
│                                                                                                                 │
│  +62                                                                                                            │
│                                                                                                                 │
│  812-3456-7890                                                                                                  │
│                                                                                                                 │
│  Email                                                                                                          │
│                                                                                                                 │
│  Address                                                                                                        │
│                                                                                                                 │
│  :                                                                                                              │
│                                                                                                                 │
│  sonny.wawwak@email.com                                                                                         │
│                                                                                                                 │
│  Skills  ●  UI  Design  (Web  &  Mobile)  ●  UX  Research  &  Analysis  ●  Wireframing  &  Prototyping  ●       │
│  Design  Systems  &  Style  Guides  ●  User  Flows  &  Journey  Mapping  ●  Usability  Testing  ●  Tools:       │
│  Figma,  Adobe  XD,  Sketch,  InVision,  Photoshop,  Il

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Candidate Screening Agent                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  MINIMAL REQUIREMENTS ANALYSIS :                                                                                │
│                                                                                                                 │
│  Requirement 1: No experience in AI lifecycle or model development.                                             │
│  Requirement 2: No experience with LLMs or ERP workflows.                                                       │
│  Requirement 3: No experience in building AI features like recommendation engines or NLP tools.                 │
│                                                                                                                 │
│  PREFERRED REQUIREMENTS ANALYSIS :                                                                              │
│  No experience with scalable cloud-based AI solutions or multi-cloud platforms.                                 │
│                                                                                                                 │
│  OVERALL MATCH PERCENTAGE :                                                                                     │
│  Approximately 10% match based on basic technical understanding (HTML, CSS).                                    │
│                                                                                                                 │
│  Strengths:                                                                                                     │
│  - Strong UI/UX design skills and experience.                                                                   │
│  - Proficient with design tools like Figma and Adobe XD.                                                        │
│                                                                                                                 │
│  Potential Gaps:                                                                                                │
│  - Lacks experience in AI, machine learning, or related technical skills.                                       │
│  - No background in cloud platforms or AI deployment.                                                           │
│                                                                                                                 │
│  RESULT : 3 of 10                                                                                               │
│                                                                                                                 │
│  (No interview questions are generated as the result is less than 7)                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ea3357ab-c8fb-41b4-97f7-3fbfe5101034                                                                     │
│  Agent: Candidate Screening Agent                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a75f7074-9bcb-4585-8710-fd3d6558ea07                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: MINIMAL REQUIREMENTS ANALYSIS :                                                                  │
│                                                                                                                 │
│  Requirement 1: No experience in AI lifecycle or model development.                                             │
│  Requirement 2: No experience with LLMs or ERP workflows.                                                       │
│  Requirement 3: No experience in building AI features like recommendation engines or NLP tools.                 │
│                                                                                                                 │
│  PREFERRED REQUIREMENTS ANALYSIS :                                                                              │
│  No experience with scalable cloud-based AI solutions or multi-cloud platforms.                                 │
│                                                                                                                 │
│  OVERALL MATCH PERCENTAGE :                                                                                     │
│  Approximately 10% match based on basic technical understanding (HTML, CSS).                                    │
│                                                                                                                 │
│  Strengths:                                                                                                     │
│  - Strong UI/UX design skills and experience.                                                                   │
│  - Proficient with design tools like Figma and Adobe XD.                                                        │
│                                                                                                                 │
│  Potential Gaps:                                                                                                │
│  - Lacks experience in AI, machine learning, or related technical skills.                                       │
│  - No background in cloud platforms or AI deployment.                                                           │
│                                                                                                                 │
│  RESULT : 3 of 10                                                                                               │
│                                                                                                                 │
│  (No interview questions are generated as the result is less than 7)                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
print(result_2)

MINIMAL REQUIREMENTS ANALYSIS :

Requirement 1: No experience in AI lifecycle or model development.
Requirement 2: No experience with LLMs or ERP workflows.
Requirement 3: No experience in building AI features like recommendation engines or NLP tools.

PREFERRED REQUIREMENTS ANALYSIS :
No experience with scalable cloud-based AI solutions or multi-cloud platforms.

OVERALL MATCH PERCENTAGE : 
Approximately 10% match based on basic technical understanding (HTML, CSS).

Strengths:
- Strong UI/UX design skills and experience.
- Proficient with design tools like Figma and Adobe XD.

Potential Gaps:
- Lacks experience in AI, machine learning, or related technical skills.
- No background in cloud platforms or AI deployment.

RESULT : 3 of 10

(No interview questions are generated as the result is less than 7)
